# **Tarea 2: Análisis de Video**

Nombre: Matías Bozo Pizarro

Rut: 20442784-4

En primera instancia, instalamos "gdown" para poder descargar las colecciones de datos desde Google Drive.

In [ ]:
!pip install gdown
!pip install unrar

Posteriormente, descargamos las colecciones de datos BDSumm y OpenVideo, desde Google Drive.

In [ ]:
# Link BDSumm.rar:    https://drive.google.com/file/d/1KQV4vDqd26wrhKguT6ER5xrV_ogQikPN/view?usp=sharing
# Link OpenVideo.rar: https://drive.google.com/file/d/12YrnmQDHG_97us_FWHd0Kce6GsG88YGj/view?usp=sharing

import gdown

# Identificar cada colección de datos, según su link de Google Drive
file_ids = [
    "1KQV4vDqd26wrhKguT6ER5xrV_ogQikPN",
    "12YrnmQDHG_97us_FWHd0Kce6GsG88YGj"
]

# Nombrar ambos archivos .rar
output_files = [
    "BDSumm.rar",
    "OpenVideo.rar"
]

# Descargar las colecciones de datos desde Google Drive
for file_id, output_file in zip(file_ids, output_files):
    gdown.download(id=file_id, output=output_file)

Downloading...
From (original): https://drive.google.com/uc?id=1KQV4vDqd26wrhKguT6ER5xrV_ogQikPN
From (redirected): https://drive.google.com/uc?id=1KQV4vDqd26wrhKguT6ER5xrV_ogQikPN&confirm=t&uuid=8cce76c3-3045-49c3-87ea-815c92db8045
To: /content/BDSumm.rar
100%|██████████| 180M/180M [00:06<00:00, 29.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=12YrnmQDHG_97us_FWHd0Kce6GsG88YGj
From (redirected): https://drive.google.com/uc?id=12YrnmQDHG_97us_FWHd0Kce6GsG88YGj&confirm=t&uuid=b4f7f7b1-248b-4f7e-a2d8-18281a39ede8
To: /content/OpenVideo.rar
100%|██████████| 367M/367M [00:08<00:00, 43.3MB/s]


Tras la descarga, procedemos a descomprimir cada archivo .rar, correspondiente a BDSumm y OpenVideo.

In [ ]:
!unrar x BDSumm.rar
!unrar x OpenVideo.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from BDSumm.rar

Creating    BDSumm                                                    OK
Extracting  BDSumm/1.jpg                                                   0%  OK 
Extracting  BDSumm/f4.fig                                                  0%  1%  OK 
Extracting  BDSumm/v11.mp4                                                 1%  2%  3%  4%  OK 
Extracting  BDSumm/v12.mp4                                                 4%  5%  6%  7%  8%  9%  OK 
Extracting  BDSumm/v13.mp4                                                 9% 10% 11% 12% 13% 14% 15% 16%  OK 
Extracting  BDSumm/v14.mp4                                                16% 17% 18% 19% 20% 21%  OK 
Extracting  BDSumm/v15.mp4                                                21% 22% 23% 2

Ahora comprobamos la extensión de los archivos presentes en cada colección de datos.

In [ ]:
import os

# Definir directorios que contienen las colecciones de datos
BDSumm_dir = "/content/BDSumm"
OpenVideo_dir = "/content/OpenVideoDB"

BDSumm_file_list = os.listdir(BDSumm_dir)
OpenVideo_file_list = os.listdir(OpenVideo_dir)

# Función para comprobar la extensión de los archivos en un directorio
def check_extensions(directory, extension):
  for filename in os.listdir(directory):
    if not filename.endswith(extension):
      print(f"Archivo {filename} no posee la extensión {extension}")
      return False
  return True

# Instanciar la función para ambas colecciones de datos
if check_extensions(BDSumm_dir, ".mp4"):
  print("Todos los archivos en BDSumm possen la extensión .mp4.")
else:
  print("No todos los archivos en BDSumm possen la extensión .mp4.")

if check_extensions(OpenVideo_dir, ".mp4"):
  print("Todos los archivos en OpenVideoDB possen la extensión .mp4.")
else:
  print("No todos los archivos en OpenVideoDB possen la extensión .mp4.")

Archivo 1.jpg no posee la extensión .mp4
No todos los archivos en BDSumm possen la extensión .mp4.
Todos los archivos en OpenVideoDB possen la extensión .mp4.


# **1) Computar los sumarios estáticos de los videos que forman parte de las colecciones BDSumm y OpenVideo usando el método K-means. Debe realizar un estudio experimental para determinar el mejor valor de K (probar con K = 3, K = 5 y K=7) para obtener los sumarios en cada colección de datos. El best-K se relaciona al mejor valor de mean-f-score alcanzado por la colección de datos. El mean-fscore es el promedio simple de f-score sobre la colección de datos completa. Mientras el f-score es la medida de comparar el sumario humano respecto del sumario automático.**

Las colecciones de datos contienen los sumarios humanos en diferentes tipos de archivos.


*   Por un lado, la colección BDSumm contiene los sumarios en archivos .txt.
*   Por otro lado, la colección Open Video contiene los sumarios en archivos .jpeg.

Para estandarizar los sumarios humanos, se obtendrán los índices de los frames del sumario humano de cada video de ambas colecciones, en diccionarios. Sin embargo, hay un obstáculo: cada colección de datos contiene sumarios humanos hechos por 3 diferentes usuarios, al menos. Para efectos de esta tarea, se obtendrán todos los sumarios humanos desde un mismo usuario (ej: "User1").

*   get_human_summaries_from_txt(), permite obtener los sumarios SOLO desde la colección de datos BDSumm.
*   get_human_summaries_from_jpeg(), permite obtener los sumarios humanos SOLO desde la colección de datos OpenVideo.

De todas maneras, el usuario del cual se pueden extraer los sumarios, se puede configurar en los parámetros de ambas funciones.








In [ ]:
# Crea un diccionario con los nombres de las carpetas (videos) como claves y los índices de frames desde los archivos de texto para un usuario específico
def get_human_summaries_from_txt(directory, user_file):
    human_summaries = {}

    try:
        # Recorrer las carpetas de videos
        for video_name in os.listdir(directory):
            video_path = os.path.join(directory, video_name)
            if os.path.isdir(video_path):  # Comprobar si es una carpeta
                user_file_path = os.path.join(video_path, user_file)

                if os.path.exists(user_file_path):
                    # Leer índices desde el archivo
                    with open(user_file_path, "r") as file:
                        frame_indices = [int(line.strip()) for line in file.readlines()]
                        human_summaries[video_name] = frame_indices
                else:
                    print(f"Archivo del usuario '{user_file}' no encontrado en {video_name}.")
    except Exception as e:
        print(f"Error al procesar el directorio: {e}")

    return human_summaries

Ahora se obtiene el sumario humano de BDSumm, con un usuario específico: US-1.

In [ ]:
# Ruta del directorio de sumario humanos de BDSumm
human_summary_BDSumm_dir = "/content/UserBDSumm"
# Obtener sumarios humanos de BDSumm
human_summaries_BDSumm = get_human_summaries_from_txt(human_summary_BDSumm_dir, user_file="US-1.txt")

# Mostrar el diccionario creado
for video, frames in human_summaries_BDSumm.items():
    print(f"Video: {video}, Frames: {frames}")

Video: v11, Frames: [21, 76, 142, 447, 540, 723, 812, 875, 909, 1030, 1078, 1148, 1339, 1419, 1478, 1725, 1789, 1875, 2047, 2134, 2264, 2320]
Video: v27, Frames: [16, 61, 94, 162, 251, 302, 373, 458, 515, 604, 680, 705, 790, 893, 941, 1026, 1096]
Video: v25, Frames: [15, 50, 81, 126, 325, 540, 776, 826, 876, 920, 1070, 1270, 1396, 1545, 1722, 1800]
Video: v20, Frames: [36, 65, 253, 352, 488, 534, 571, 807, 838, 882, 979, 1027, 1123, 1180, 1228, 1257, 1278, 1346, 1405, 1462, 1547, 1598, 1624, 1696, 1754, 1850, 1933, 1977, 2035, 2092, 2110, 2211, 2352, 2492, 2683, 3010, 3095]
Video: v21, Frames: [18, 49, 94, 179, 257, 298, 344, 390, 431, 611, 667, 755, 850, 944, 991, 1045, 1210, 1297, 1330]
Video: v17, Frames: [32, 70, 352, 423, 576, 757, 885, 1023, 1454, 1540, 1692, 2005, 2159, 2304, 2448, 2549, 2791, 2862, 3075, 3225, 3589, 3662, 3804, 3944, 4097, 4131, 4204, 4308, 4382, 4505, 4793, 4986, 5037]
Video: v15, Frames: [16, 97, 241, 413, 532, 746, 832, 1026, 1399, 1491, 1577, 2028, 2124, 21

Ahora se obtiene el sumario humano de OpenVideo, con un usuario específico: user-1.

In [ ]:
# Crea un diccionario con los nombres de las carpetas (videos) como claves y los índices de frames desde el nombre de una imagen para un usuario específico
def get_human_summaries_from_jpeg(directory, user_folder):
    human_summaries = {}

    try:
        # Recorrer las carpetas de videos
        for video_name in os.listdir(directory):
            video_path = os.path.join(directory, video_name)
            if os.path.isdir(video_path):  # Comprobar que sea una carpeta
                user_path = os.path.join(video_path, user_folder)

                if os.path.exists(user_path) and os.path.isdir(user_path):
                    # Obtener nombres de archivos .jpeg en la carpeta del usuario
                    frame_files = [f for f in os.listdir(user_path) if f.endswith(".jpeg")]

                    # Extraer los índices de los nombres de los archivos
                    frame_indices = [int(f.replace("Frame", "").replace(".jpeg", "")) for f in frame_files]

                    # Guardar los índices en el diccionario
                    human_summaries[video_name] = sorted(frame_indices)  # Ordenar los índices
                else:
                    print(f"Carpeta del usuario '{user_folder}' no encontrada en {video_name}.")
    except Exception as e:
        print(f"Error al procesar el directorio: {e}")

    return human_summaries

In [ ]:
# Ruta del directorio de sumario humanos de OpenVideo
human_summary_OpenVideo_dir = "/content/UserOpenVideoSummary"
# Obtener sumarios humanos de OpenVideo
human_summaries_OpenVideo = get_human_summaries_from_jpeg(human_summary_OpenVideo_dir, user_folder="user1")

# Mostrar el diccionario creado
for video, frames in human_summaries_OpenVideo.items():
    print(f"Video: {video}, Frames: {frames}")

Video: v58, Frames: [391, 841, 1171, 1681, 1981, 3031]
Video: v31, Frames: [181, 361, 510, 691, 781, 871, 1141, 1322, 1471, 1921, 2071, 2251, 2371]
Video: v64, Frames: [121, 361, 511, 871, 1711, 2521, 2791, 2881, 3061, 3331, 3631, 4111, 4681, 5011]
Video: v67, Frames: [31, 331, 661, 871, 1021, 1891, 2191]
Video: v42, Frames: [180, 691, 811, 1471, 1621, 2251, 2551]
Video: v27, Frames: [451, 991, 1201, 1411, 1711, 2311]
Video: v54, Frames: [451, 691, 841, 1231, 1501, 1561, 1621]
Video: v25, Frames: [151, 601, 661, 811, 991, 1081, 1171, 1201, 1381, 1471, 1561, 1621, 1711]
Video: v44, Frames: [31, 301, 781, 991, 1021, 1171, 1351, 1381, 1590, 1621, 1741, 2071]
Video: v33, Frames: [301, 481, 601, 841, 991, 1471, 1651, 1771, 1951, 2161, 2311, 2341, 2551, 2821, 3061, 3151]
Video: v52, Frames: [91, 961, 1351, 1681, 2251, 2791, 2971, 3511]
Video: v38, Frames: [61, 152, 811, 1891, 2011, 2221, 2611, 2911, 3511, 3721, 4112]
Video: v68, Frames: [31, 301, 511, 841, 1861]
Video: v59, Frames: [451, 481

Importamos OpenCV para leer cada video y obtener sus frames.

In [ ]:
import cv2

Debido al alto consumo de memoria RAM que implica la lectura de todos los frames de todos los videos, se implementa un prefiltrado, donde se selecciona solo un frame por segundo, basado en la tasa de cuadros por segundo (FPS) del respectivo video.

PD: Se intentaron diversas estrategias para leer todos los frames de todos los videos, sin embargo la RAM colapsó en todos los intentos.

In [ ]:
# Extrae un porcentaje específico de los frames de cada video en un directorio y los guarda en un diccionario
def extract_frames_from_directory(directory, percentage):
    if not (1 <= percentage <= 100):
        raise ValueError("El porcentaje debe estar entre 1 y 100")

    # Calcular el factor de subsampling según el porcentaje
    factor = int(100 / percentage)

    frames_dictionary = {}

    for filename in os.listdir(directory):
        # Procesar solo archivos .mp4
        if filename.endswith(".mp4"):
            video_path = os.path.join(directory, filename)
            vid_capture = cv2.VideoCapture(video_path)

            if not vid_capture.isOpened():
                print(f"Error al leer el video: {filename}")
                continue

            # Inicializar lista de frames para un video
            video_name = os.path.splitext(filename)[0]
            frames_dictionary[video_name] = []

            frame_count = 0

            while True:
                # Leer frame
                ret, frame = vid_capture.read()
                if not ret:
                    # Salir si no hay más frames
                    break

                # Extraer un frame según el factor de subsampling
                if frame_count % factor == 0:
                    # Convertir frame a RGB
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Guardar frame en el diccionario
                    frames_dictionary[video_name].append(frame_rgb)

                frame_count += 1

            vid_capture.release()
            print(f"Frames de {video_name} extraídos y guardados en el diccionario. Total frames extraídos: {len(frames_dictionary[video_name])}")

    return frames_dictionary

Se extrae el 10% del total de los frames de cada video de ambas colecciones de datos

PD: Se intentó con 50%, 30%, 20% y 15%, sin embargo el consumo de memoria RAM superó el límite

In [ ]:
BDSumm_dict = extract_frames_from_directory(BDSumm_dir, 10)

Frames de v23 extraídos y guardados en el diccionario. Total frames extraídos: 505
Frames de v19 extraídos y guardados en el diccionario. Total frames extraídos: 265
Frames de v15 extraídos y guardados en el diccionario. Total frames extraídos: 256
Frames de v11 extraídos y guardados en el diccionario. Total frames extraídos: 235
Frames de v25 extraídos y guardados en el diccionario. Total frames extraídos: 183
Frames de v12 extraídos y guardados en el diccionario. Total frames extraídos: 151
Frames de v27 extraídos y guardados en el diccionario. Total frames extraídos: 116
Frames de v21 extraídos y guardados en el diccionario. Total frames extraídos: 134
Frames de v20 extraídos y guardados en el diccionario. Total frames extraídos: 310
Frames de v16 extraídos y guardados en el diccionario. Total frames extraídos: 265
Frames de v30 extraídos y guardados en el diccionario. Total frames extraídos: 263
Frames de v18 extraídos y guardados en el diccionario. Total frames extraídos: 345
Fram

In [ ]:
OpenVideo_dict = extract_frames_from_directory(OpenVideo_dir, 10)

Frames de v23 extraídos y guardados en el diccionario. Total frames extraídos: 176
Frames de v50 extraídos y guardados en el diccionario. Total frames extraídos: 483
Frames de v59 extraídos y guardados en el diccionario. Total frames extraídos: 211
Frames de v33 extraídos y guardados en el diccionario. Total frames extraídos: 327
Frames de v68 extraídos y guardados en el diccionario. Total frames extraídos: 195
Frames de v25 extraídos y guardados en el diccionario. Total frames extraídos: 180
Frames de v46 extraídos y guardados en el diccionario. Total frames extraídos: 360
Frames de v64 extraídos y guardados en el diccionario. Total frames extraídos: 531
Frames de v56 extraídos y guardados en el diccionario. Total frames extraídos: 233
Frames de v37 extraídos y guardados en el diccionario. Total frames extraídos: 342
Frames de v39 extraídos y guardados en el diccionario. Total frames extraídos: 525
Frames de v48 extraídos y guardados en el diccionario. Total frames extraídos: 371
Fram

Hay una observación relevante en la extracción de frames. Por un lado, los sumarios humanos consideran el total de frames, mientras que la extracción realizada en este programa, solo considera el 10%. Es necesario que los frames se encuentren en la misma escala. En consecuencia, se implementa una función para normalizar los frames del sumario.

In [ ]:
# Normaliza los índices de los sumarios humanos basados en el porcentaje de frames extraídos
def normalize_human_summaries(summaries, video_frames, percentage):
    if not (1 <= percentage <= 100):
        raise ValueError("El porcentaje debe estar entre 1 y 100")

    # Factor de normalización basado en el porcentaje
    factor = 100 / percentage
    normalized_summaries = {}

    for video_name, frame_indices in summaries.items():
        if video_name in video_frames:
            total_extracted_frames = len(video_frames[video_name])  # Total de frames extraídos

            normalized_indices = []
            for index in frame_indices:
                # Calcular índice normalizado
                normalized_index = int(index / factor)

                # Asegurarse de que el índice no exceda los frames extraídos
                if normalized_index < total_extracted_frames:
                    normalized_indices.append(normalized_index)

            normalized_summaries[video_name] = normalized_indices
        else:
            print(f"Advertencia: Video '{video_name}' no encontrado en directorio.")

    return normalized_summaries

Se aplica la función de normalización según el porcentaje definido anteriormente (10%), para ambas colecciones de datos.

In [ ]:
normalized_summaries_BDSumm = normalize_human_summaries(human_summaries_BDSumm, BDSumm_dict, 10)

for video, frames in normalized_summaries_BDSumm.items():
    print(f"Video: {video}, Normalized Frames: {frames}")

Video: v11, Normalized Frames: [2, 7, 14, 44, 54, 72, 81, 87, 90, 103, 107, 114, 133, 141, 147, 172, 178, 187, 204, 213, 226, 232]
Video: v27, Normalized Frames: [1, 6, 9, 16, 25, 30, 37, 45, 51, 60, 68, 70, 79, 89, 94, 102, 109]
Video: v25, Normalized Frames: [1, 5, 8, 12, 32, 54, 77, 82, 87, 92, 107, 127, 139, 154, 172, 180]
Video: v20, Normalized Frames: [3, 6, 25, 35, 48, 53, 57, 80, 83, 88, 97, 102, 112, 118, 122, 125, 127, 134, 140, 146, 154, 159, 162, 169, 175, 185, 193, 197, 203, 209, 211, 221, 235, 249, 268, 301, 309]
Video: v21, Normalized Frames: [1, 4, 9, 17, 25, 29, 34, 39, 43, 61, 66, 75, 85, 94, 99, 104, 121, 129, 133]
Video: v17, Normalized Frames: [3, 7, 35, 42, 57, 75, 88, 102, 145, 154, 169, 200, 215, 230, 244, 254, 279, 286, 307, 322, 358, 366, 380, 394, 409, 413, 420, 430, 438, 450, 479, 498, 503]
Video: v15, Normalized Frames: [1, 9, 24, 41, 53, 74, 83, 102, 139, 149, 157, 202, 212, 216, 229, 245, 250]
Video: v28, Normalized Frames: [1, 2, 9, 18, 22, 29, 36, 46, 5

In [ ]:
normalized_summaries_OpenVideo = normalize_human_summaries(human_summaries_OpenVideo, OpenVideo_dict, 10)

for video, frames in normalized_summaries_OpenVideo.items():
    print(f"Video: {video}, Normalized Frames: {frames}")

Video: v58, Normalized Frames: [39, 84, 117, 168, 198, 303]
Video: v31, Normalized Frames: [18, 36, 51, 69, 78, 87, 114, 132, 147, 192, 207, 225, 237]
Video: v64, Normalized Frames: [12, 36, 51, 87, 171, 252, 279, 288, 306, 333, 363, 411, 468, 501]
Video: v67, Normalized Frames: [3, 33, 66, 87, 102, 189, 219]
Video: v42, Normalized Frames: [18, 69, 81, 147, 162, 225, 255]
Video: v27, Normalized Frames: [45, 99, 120, 141, 171, 231]
Video: v54, Normalized Frames: [45, 69, 84, 123, 150, 156, 162]
Video: v25, Normalized Frames: [15, 60, 66, 81, 99, 108, 117, 120, 138, 147, 156, 162, 171]
Video: v44, Normalized Frames: [3, 30, 78, 99, 102, 117, 135, 138, 159, 162, 174, 207]
Video: v33, Normalized Frames: [30, 48, 60, 84, 99, 147, 165, 177, 195, 216, 231, 234, 255, 282, 306, 315]
Video: v52, Normalized Frames: [9, 96, 135, 168, 225, 279, 297, 351]
Video: v38, Normalized Frames: [6, 15, 81, 189, 201, 222, 261, 291, 351, 372, 411]
Video: v68, Normalized Frames: [3, 30, 51, 84, 186]
Video: v59,

Para describir los frames leídos, haremos uso de Pytorch para utilizar una red neuronal convolucional.

In [ ]:
import torch #Librería Pytorch
import torchvision #Módulo para visión computacional de Pytorch
import torchvision.transforms as T #Módulo para transformaciones de imágenes
from PIL import Image #Módulo para leer imágenes
import matplotlib.pyplot as plt
import numpy as np

Estas transformaciones preparan el input para la red neuronal. En este caso se aplican las transformaciones:


*   Resize(256): redimensiona el frame de tal manera que el lado más corto tenga 256 píxeles.
*   CenterCrop(224): realiza un recorte de un parche de 224x224.
*   ToTensor: Convierte el frame a un tensor de Pytorch y normaliza sus valores entre 0 y 1.
*   Normalize: normaliza la distribución de los píxeles de acuerdo a la estadística de ImageNet (dataset con el que fue entrenada la red que usaremos).



In [ ]:
transform=T.Compose([T.Resize(256),T.CenterCrop(224),T.ToTensor(),T.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

Cargamos una ResNet18 preentrenada. La ponemos en modo evaluación, ya que solo haremos inferencias. Enviamos la red al GPU.

In [ ]:
#Cargar modelo pre-entrenado
model = torchvision.models.resnet18(pretrained=True)

#Habilitar el modo de evaluación que mejora el performance para la inferencia
model.eval()
model.to('cuda') #Mandamos el modelo al GPU

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 82.6MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

A partir de cada frame de cada video contenido en el diccionario, se extraen los descriptores mediante ResNet-18. Se utiliza ResNet-18 en base a su rendimiento, que supera otros métodos como la obtención de descriptores mediante HOG y Color.

In [ ]:
# Extrae descriptores para los frames de cada video y los almacena en un diccionario.
def extract_descriptors_dict(frames_dict, model, transform):
    descriptors_dict = {}  # Diccionario para almacenar los descriptores
    model.eval()  # Configurar modelo en modo evaluación
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Enviar modelo a GPU/CPU

    with torch.no_grad():  # No calcular gradientes
        for video_name, frames in frames_dict.items():
            video_descriptors = []  # Lista para los descriptores de este video

            for frame in frames:
                # Convertir el frame a formato PIL y aplicar las transformaciones
                pil_image = Image.fromarray(frame)
                input_tensor = transform(pil_image).unsqueeze(0).to(device)  # Agregar dimensión batch

                # Pasar el frame a través del modelo
                output = model(input_tensor)
                descriptor = output.cpu().numpy().flatten()  # Aplanar y convertir a numpy
                video_descriptors.append(descriptor)

            # Guardar los descriptores de este video en el diccionario
            descriptors_dict[video_name] = np.array(video_descriptors)

    return descriptors_dict

Ahora se extraen los descriptores para cada video de ambas colecciones de datos.

In [ ]:
# Extraer descriptores para cada video de BDSumm
descriptors_by_video = extract_descriptors_dict(BDSumm_dict, model, transform)

# Verificar el número de descriptores extraídos por video
for video_name, descriptors in descriptors_by_video.items():
    print(f"Video: {video_name}, Descriptores extraídos: {descriptors.shape}")

Video: v23, Descriptores extraídos: (505, 1000)
Video: v19, Descriptores extraídos: (265, 1000)
Video: v15, Descriptores extraídos: (256, 1000)
Video: v11, Descriptores extraídos: (235, 1000)
Video: v25, Descriptores extraídos: (183, 1000)
Video: v12, Descriptores extraídos: (151, 1000)
Video: v27, Descriptores extraídos: (116, 1000)
Video: v21, Descriptores extraídos: (134, 1000)
Video: v20, Descriptores extraídos: (310, 1000)
Video: v16, Descriptores extraídos: (265, 1000)
Video: v30, Descriptores extraídos: (263, 1000)
Video: v18, Descriptores extraídos: (345, 1000)
Video: v29, Descriptores extraídos: (158, 1000)
Video: v24, Descriptores extraídos: (334, 1000)
Video: v26, Descriptores extraídos: (123, 1000)
Video: v13, Descriptores extraídos: (392, 1000)
Video: v28, Descriptores extraídos: (94, 1000)
Video: v22, Descriptores extraídos: (318, 1000)
Video: v17, Descriptores extraídos: (509, 1000)
Video: v14, Descriptores extraídos: (346, 1000)


In [ ]:
# Extraer descriptores para cada video
descriptors_by_video_OpenVideo = extract_descriptors_dict(OpenVideo_dict, model, transform)

# Verificar el número de descriptores extraídos por video
for video_name, descriptors in descriptors_by_video_OpenVideo.items():
    print(f"Video: {video_name}, Descriptores extraídos: {descriptors.shape}")

Video: v23, Descriptores extraídos: (176, 1000)
Video: v50, Descriptores extraídos: (483, 1000)
Video: v59, Descriptores extraídos: (211, 1000)
Video: v33, Descriptores extraídos: (327, 1000)
Video: v68, Descriptores extraídos: (195, 1000)
Video: v25, Descriptores extraídos: (180, 1000)
Video: v46, Descriptores extraídos: (360, 1000)
Video: v64, Descriptores extraídos: (531, 1000)
Video: v56, Descriptores extraídos: (233, 1000)
Video: v37, Descriptores extraídos: (342, 1000)
Video: v39, Descriptores extraídos: (525, 1000)
Video: v48, Descriptores extraídos: (371, 1000)
Video: v54, Descriptores extraídos: (289, 1000)
Video: v32, Descriptores extraídos: (269, 1000)
Video: v60, Descriptores extraídos: (210, 1000)
Video: v27, Descriptores extraídos: (319, 1000)
Video: v69, Descriptores extraídos: (362, 1000)
Video: v52, Descriptores extraídos: (362, 1000)
Video: v21, Descriptores extraídos: (329, 1000)
Video: v61, Descriptores extraídos: (227, 1000)
Video: v51, Descriptores extraídos: (294

Ahora, usaremos el K-Means que viene implementado en Scikit Learn.

In [ ]:
from sklearn.cluster import KMeans

Aquí implementamos la función que hace el clustering de los descriptores que están almacenados en los diccionarios y selecciona los keyframes a partir de su distancia a los centroides.

In [ ]:
# Aplica K-means a los descriptores de cada video y selecciona los keyframes más representativos
def apply_kmeans_to_videos(descriptors_dict, k):
    keyframes_by_video = {}

    for video_name, descriptors in descriptors_dict.items():
        # Aplicar K-means
        kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto").fit(descriptors)

        # Calcular la distancia de cada descriptor al centroide de su cluster
        distances = np.linalg.norm(descriptors[:, None] - kmeans.cluster_centers_, axis=2)
        closest_indices = np.argmin(distances, axis=0)  # Índices más cercanos a los centroides

        # Guardar los índices de los keyframes seleccionados
        keyframes_by_video[video_name] = closest_indices

    return keyframes_by_video

Podemos aplicar k-means según la cantidad de clústers solicitados en la tarea (k=3, k=5 y k=7), para ambas colecciones de datos.

In [ ]:
# Aplicar k-means en BDSumm
BDSumm_kmeans_3 = apply_kmeans_to_videos(descriptors_by_video, k=3)
BDSumm_kmeans_5 = apply_kmeans_to_videos(descriptors_by_video, k=5)
BDSumm_kmeans_7 = apply_kmeans_to_videos(descriptors_by_video, k=7)

In [ ]:
# Aplicar k-means en OpenVideo
OpenVideo_kmeans_3 = apply_kmeans_to_videos(descriptors_by_video_OpenVideo, k=3)
OpenVideo_kmeans_5 = apply_kmeans_to_videos(descriptors_by_video_OpenVideo, k=5)
OpenVideo_kmeans_7 = apply_kmeans_to_videos(descriptors_by_video_OpenVideo, k=7)

Como inspección preliminar, observamos los keyframes para la colección de datos OpenVideo, para k=3, k=5 y k=7

In [ ]:
for video_name, keyframes in OpenVideo_kmeans_3.items():
    print(f"Video: {video_name}, Keyframes seleccionados: {keyframes}")

Video: v23, Keyframes seleccionados: [ 43 111  76]
Video: v50, Keyframes seleccionados: [ 91 428  62]
Video: v59, Keyframes seleccionados: [ 79 189  32]
Video: v33, Keyframes seleccionados: [ 41 114  15]
Video: v68, Keyframes seleccionados: [ 52 125   8]
Video: v25, Keyframes seleccionados: [102 111  33]
Video: v46, Keyframes seleccionados: [280 325 341]
Video: v64, Keyframes seleccionados: [190 530 159]
Video: v56, Keyframes seleccionados: [112 211  32]
Video: v37, Keyframes seleccionados: [318 170  16]
Video: v39, Keyframes seleccionados: [111 359 414]
Video: v48, Keyframes seleccionados: [143 109  40]
Video: v54, Keyframes seleccionados: [111 215   3]
Video: v32, Keyframes seleccionados: [167 250  76]
Video: v60, Keyframes seleccionados: [197 201  31]
Video: v27, Keyframes seleccionados: [145 173   0]
Video: v69, Keyframes seleccionados: [237 320 205]
Video: v52, Keyframes seleccionados: [299 178  37]
Video: v21, Keyframes seleccionados: [128  11  49]
Video: v61, Keyframes seleccion

In [ ]:
for video_name, keyframes in OpenVideo_kmeans_5.items():
    print(f"Video: {video_name}, Keyframes seleccionados: {keyframes}")

Video: v23, Keyframes seleccionados: [ 65 114  23 162  43]
Video: v50, Keyframes seleccionados: [200 428  62 259 101]
Video: v59, Keyframes seleccionados: [ 79 185  32 137  87]
Video: v33, Keyframes seleccionados: [126 114 180 161  15]
Video: v68, Keyframes seleccionados: [ 52  16   8  80 182]
Video: v25, Keyframes seleccionados: [102 172  33 126 147]
Video: v46, Keyframes seleccionados: [144 321  52 249 349]
Video: v64, Keyframes seleccionados: [190 530  65 405 154]
Video: v56, Keyframes seleccionados: [ 84 218  32 113 163]
Video: v37, Keyframes seleccionados: [142 170  16 271 303]
Video: v39, Keyframes seleccionados: [111 123  36 414 521]
Video: v48, Keyframes seleccionados: [143 106  40 356 304]
Video: v54, Keyframes seleccionados: [111 215  46  65  88]
Video: v32, Keyframes seleccionados: [ 43  71  12 167 250]
Video: v60, Keyframes seleccionados: [169 201  31 134  18]
Video: v27, Keyframes seleccionados: [317 173  36 145 137]
Video: v69, Keyframes seleccionados: [218 277  19 232 33

In [ ]:
for video_name, keyframes in OpenVideo_kmeans_7.items():
    print(f"Video: {video_name}, Keyframes seleccionados: {keyframes}")

Video: v23, Keyframes seleccionados: [ 65 112  23 162  43  35 100]
Video: v50, Keyframes seleccionados: [312 428  62 259 127 101 378]
Video: v59, Keyframes seleccionados: [ 79 185  32 137  87  60 107]
Video: v33, Keyframes seleccionados: [126 288 178   6  28  77 114]
Video: v68, Keyframes seleccionados: [ 67 131   8  80 106  16  52]
Video: v25, Keyframes seleccionados: [102 172  20 122 147  39  98]
Video: v46, Keyframes seleccionados: [144 321  52 249 357 117 167]
Video: v64, Keyframes seleccionados: [190 451  65 420 393 530 154]
Video: v56, Keyframes seleccionados: [ 86 218  32 113 163  53  76]
Video: v37, Keyframes seleccionados: [142 170  16 227 306  40 231]
Video: v39, Keyframes seleccionados: [111 245  36 421 521   3 155]
Video: v48, Keyframes seleccionados: [212 229   6 356 304  40  89]
Video: v54, Keyframes seleccionados: [111 215  46 156 150  65 103]
Video: v32, Keyframes seleccionados: [ 43 223  12 167 240  73 121]
Video: v60, Keyframes seleccionados: [ 77 154  31 134  18  56 

Ahora es necesario determinar el mejor valor de k (best-k), para lo cual se calculará el F-score. A continuación, se implementan las funciones necesarias para llevar a cabo este cálculo.

In [ ]:
# Calcula la precisión y el recall basados en los índices predichos (k-means) y los índices humanos (sumarios).
def calculate_precision_recall(predicted_indices, human_indices):
    # Convertir listas a conjuntos para operaciones de intersección
    predicted_set = set(predicted_indices)
    human_set = set(human_indices)

    # Calcular True Positives (TP)
    tp = len(predicted_set & human_set)

    # Calcular False Positives (FP) y False Negatives (FN)
    fp = len(predicted_set - human_set)  # Predichos, pero no están en los humanos
    fn = len(human_set - predicted_set)  # Humanos, pero no están en los predichos

    # Calcular precisión y recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

# Calcula el F1-score basado en la precisión y el recall.
def calculate_f_score(precision, recall):
    if precision + recall > 0:
        f_score = 2 * (precision * recall) / (precision + recall)
    else:
        f_score = 0
    return f_score

# Calcula el F1-score para cada video basado en los keyframes seleccionados y los sumarios humanos
def calculate_f_score_for_videos(predicted_keyframes, human_summaries):
    f_scores = {}

    for video_name, predicted_indices in predicted_keyframes.items():
        if video_name in human_summaries:
            human_indices = human_summaries[video_name]
            precision, recall = calculate_precision_recall(predicted_indices, human_indices)
            f_score = calculate_f_score(precision, recall)
            f_scores[video_name] = f_score
        else:
            print(f"Advertencia: No se encontró un sumario humano para el video '{video_name}'")
            f_scores[video_name] = 0  # Asignar F-score 0 si no hay sumario humano

    return f_scores


A continuación, se calculan los F-scores para cada sumario de cada video, considerando los tres clusterings realizados en BDSumm.

In [ ]:
# Obtener f-scores para k-means con k=3, en BDSumm
f_scores_BDSumm_3 = calculate_f_score_for_videos(BDSumm_kmeans_3, normalized_summaries_BDSumm)

for video_name, f_score in f_scores_BDSumm_3.items():
    print(f"K=3. Video: {video_name}, F-score: {f_score}")

# Obtener f-scores para k-means con k=5, en BDSumm
f_scores_BDSumm_5 = calculate_f_score_for_videos(BDSumm_kmeans_5, normalized_summaries_BDSumm)

for video_name, f_score in f_scores_BDSumm_5.items():
    print(f"K=5. Video: {video_name}, F-score: {f_score}")

# Obtener f-scores para k-means con k=7, en BDSumm
f_score_BDSumm_7 = calculate_f_score_for_videos(BDSumm_kmeans_7, normalized_summaries_BDSumm)

for video_name, f_score in f_score_BDSumm_7.items():
    print(f"K=7. Video: {video_name}, F-score: {f_score}")

K=3. Video: v23, F-score: 0
K=3. Video: v19, F-score: 0
K=3. Video: v15, F-score: 0
K=3. Video: v11, F-score: 0
K=3. Video: v25, F-score: 0
K=3. Video: v12, F-score: 0.08
K=3. Video: v27, F-score: 0
K=3. Video: v21, F-score: 0.09090909090909091
K=3. Video: v20, F-score: 0
K=3. Video: v16, F-score: 0
K=3. Video: v30, F-score: 0
K=3. Video: v18, F-score: 0
K=3. Video: v29, F-score: 0.08
K=3. Video: v24, F-score: 0
K=3. Video: v26, F-score: 0.1111111111111111
K=3. Video: v13, F-score: 0.09523809523809525
K=3. Video: v28, F-score: 0
K=3. Video: v22, F-score: 0.05555555555555555
K=3. Video: v17, F-score: 0
K=3. Video: v14, F-score: 0
K=5. Video: v23, F-score: 0
K=5. Video: v19, F-score: 0
K=5. Video: v15, F-score: 0
K=5. Video: v11, F-score: 0
K=5. Video: v25, F-score: 0
K=5. Video: v12, F-score: 0
K=5. Video: v27, F-score: 0
K=5. Video: v21, F-score: 0.08333333333333333
K=5. Video: v20, F-score: 0
K=5. Video: v16, F-score: 0
K=5. Video: v30, F-score: 0.0588235294117647
K=5. Video: v18, F-s

A continuación, se calculan los F-scores para cada sumario de cada video, considerando los tres clusterings realizados en OpenVideo.

In [ ]:
# Obtener f-scores para k-means con k=3, en OpenVideo
f_scores_OpenVideo_3 = calculate_f_score_for_videos(OpenVideo_kmeans_3, normalized_summaries_OpenVideo)

for video_name, f_score in f_scores_OpenVideo_3.items():
    print(f"K=3. Video: {video_name}, F-score: {f_score}")

# Obtener f-scores para k-means con k=3, en OpenVideo
f_scores_OpenVideo_5 = calculate_f_score_for_videos(OpenVideo_kmeans_5, normalized_summaries_OpenVideo)

for video_name, f_score in f_scores_OpenVideo_5.items():
    print(f"K=5. Video: {video_name}, F-score: {f_score}")

# Obtener f-scores para k-means con k=3, en OpenVideo
f_scores_OpenVideo_7 = calculate_f_score_for_videos(OpenVideo_kmeans_7, normalized_summaries_OpenVideo)

for video_name, f_score in f_scores_OpenVideo_7.items():
    print(f"K=7. Video: {video_name}, F-score: {f_score}")

K=3. Video: v23, F-score: 0
K=3. Video: v50, F-score: 0
K=3. Video: v59, F-score: 0
K=3. Video: v33, F-score: 0
K=3. Video: v68, F-score: 0
K=3. Video: v25, F-score: 0
K=3. Video: v46, F-score: 0
K=3. Video: v64, F-score: 0
K=3. Video: v56, F-score: 0
K=3. Video: v37, F-score: 0
K=3. Video: v39, F-score: 0
K=3. Video: v48, F-score: 0
K=3. Video: v54, F-score: 0
K=3. Video: v32, F-score: 0
K=3. Video: v60, F-score: 0
K=3. Video: v27, F-score: 0
K=3. Video: v69, F-score: 0
K=3. Video: v52, F-score: 0
K=3. Video: v21, F-score: 0
K=3. Video: v61, F-score: 0
K=3. Video: v51, F-score: 0
K=3. Video: v42, F-score: 0
K=3. Video: v66, F-score: 0
K=3. Video: v41, F-score: 0
K=3. Video: v62, F-score: 0
K=3. Video: v34, F-score: 0
K=3. Video: v30, F-score: 0
K=3. Video: v38, F-score: 0
K=3. Video: v40, F-score: 0
K=3. Video: v57, F-score: 0
K=3. Video: v65, F-score: 0
K=3. Video: v45, F-score: 0
K=3. Video: v70, F-score: 0
K=3. Video: v29, F-score: 0
K=3. Video: v63, F-score: 0
K=3. Video: v35, F-s

Con los F-scores calculados, se procede a obtener el mean F-score, que corresponde al promedio simple de los valores obtenidos.

In [ ]:
# Calcula el mean F-score de una colección de videos
def calculate_mean_f_score(f_scores):
    if not f_scores:
        return 0  # Si no hay F-scores, el promedio es 0

    total_f_score = sum(f_scores.values())
    mean_f_score = total_f_score / len(f_scores)
    return mean_f_score

In [ ]:
# Calcular F-scores por video de cada clustering realizado (BDSumm)
mean_f_score_BDSumm_3 = calculate_mean_f_score(f_scores_BDSumm_3)
mean_f_score_BDSumm_5 = calculate_mean_f_score(f_scores_BDSumm_5)
mean_f_score_BDSumm_7 = calculate_mean_f_score(f_score_BDSumm_7)

print(f"Mean F-score para k=3 (BDSumm): {mean_f_score_BDSumm_3}")
print(f"Mean F-score para k=5 (BDSumm): {mean_f_score_BDSumm_5}")
print(f"Mean F-score para k=7 (BDSumm): {mean_f_score_BDSumm_7}")

Mean F-score para k=3 (BDSumm): 0.02564069264069264
Mean F-score para k=5 (BDSumm): 0.012663398692810458
Mean F-score para k=7 (BDSumm): 0.038784946437869984


In [ ]:
# Calcular F-scores por video de cada clustering realizado (Open Video)
mean_f_score_OpenVideo_3 = calculate_mean_f_score(f_scores_OpenVideo_3)
mean_f_score_OpenVideo_5 = calculate_mean_f_score(f_scores_OpenVideo_5)
mean_f_score_OpenVideo_7 = calculate_mean_f_score(f_scores_OpenVideo_7)

print(f"Mean F-score para k=3 (OpenVideo): {mean_f_score_OpenVideo_3}")
print(f"Mean F-score para k=5 (OpenVideo): {mean_f_score_OpenVideo_5}")
print(f"Mean F-score para k=7 (OpenVideo): {mean_f_score_OpenVideo_7}")

Mean F-score para k=3 (OpenVideo): 0.0
Mean F-score para k=5 (OpenVideo): 0.016963849016480598
Mean F-score para k=7 (OpenVideo): 0.02838224520577462


Para ambas colecciones de datos, se observa que el mean F-score alcanza su valor máximo cuando k = 7. Sin embargo, los resultados obtenidos son considerablemente bajos. Esto podría deberse a problemas relacionados con la normalización, que podrían provocar una desalineación entre los índices normalizados de los resúmenes generados por humanos y los fotogramas seleccionados automáticamente. Esto significa que los índices de los keyframes generados podrían no corresponder adecuadamente con los frames relevantes del sumario humano, disminuyendo así la precisión y el recall en el cálculo del F-score.

En consecuencia, se desarrolla un método más visual para calcular el F-score. Para ello, es necesario extraer los fotogramas (imágenes) correspondientes a los índices, lo que requiere la implementación de la función select_frames_from_indices():

In [ ]:
# Selecciona frames (imágenes) a partir de los sumarios normalizados (índice de frame -> frame).
def select_frames_from_indices(normalized_indices, frames_dict):
    selected_frames = {}
    for video_name, indices in normalized_indices.items():
        if video_name in frames_dict:
            video_frames = frames_dict[video_name]
            selected_frames[video_name] = [video_frames[i] for i in indices if i < len(video_frames)]
        else:
            print(f"Advertencia: Video '{video_name}' no encontrado en el diccionario de frames.")
    return selected_frames

Aplicamos la función a los sumarios normalizados y los keyframes obtenidos desde k-means, para ambas colecciones de datos.

In [ ]:
# Obtener frames a partir de los índices de los sumarios humanos
summaries_images_BDSumm = select_frames_from_indices(normalized_summaries_BDSumm, BDSumm_dict)
summaries_images_OpenVideo = select_frames_from_indices(normalized_summaries_OpenVideo, OpenVideo_dict)

# Obtener frames a partir de los índices de keyframes generados por k-means con k=7
key_frames_images_BDSumm_7 = select_frames_from_indices(BDSumm_kmeans_7, BDSumm_dict)
key_frames_images_OpenVideo_7 = select_frames_from_indices(OpenVideo_kmeans_7, OpenVideo_dict)

# Obtener frames a partir de los índices de keyframes generados por k-means con k=5
key_frames_images_BDSumm_5 = select_frames_from_indices(BDSumm_kmeans_5, BDSumm_dict)
key_frames_images_OpenVideo_5 = select_frames_from_indices(OpenVideo_kmeans_5, OpenVideo_dict)

# Obtener frames a partir de los índices de keyframes generados por k-means con k=3
key_frames_images_BDSumm_3 = select_frames_from_indices(BDSumm_kmeans_3, BDSumm_dict)
key_frames_images_OpenVideo_3 = select_frames_from_indices(OpenVideo_kmeans_3, OpenVideo_dict)

Con las imágenes necesarias ya obtenidas, se procede a calcular el F-score visual utilizando la función calculate_visual_f_scores_from_images().

In [ ]:
from skimage.metrics import structural_similarity as ssim

# Calcula el F1-score visual comparando imágenes de keyframes predichos con imágenes de sumarios humanos
def calculate_visual_f_scores_from_images(predicted_images, human_images, similarity_threshold):

    # Compara dos imágenes usando SSIM (Structural Similarity Index)
    def compare_images(image1, image2):
        gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        score, _ = ssim(gray1, gray2, full=True)
        return score >= similarity_threshold

    f_scores = {}

    for video_name, pred_images in predicted_images.items():
        if video_name in human_images:
            human_imgs = human_images[video_name]

            tp = 0  # True Positives
            fp = 0  # False Positives
            fn = 0  # False Negatives

            matched_human_indices = set()

            # Comparar keyframes predichos con los sumarios humanos
            for pred_image in pred_images:
                matched = False
                for i, human_image in enumerate(human_imgs):
                    if i not in matched_human_indices and compare_images(pred_image, human_image):
                        tp += 1
                        matched_human_indices.add(i)
                        matched = True
                        break
                if not matched:
                    fp += 1

            # False Negatives: Sumarios humanos no emparejados
            fn = len(human_imgs) - len(matched_human_indices)

            # Calcular Precision y Recall
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0

            # Calcular F-score
            if precision + recall > 0:
                f_score = 2 * (precision * recall) / (precision + recall)
            else:
                f_score = 0

            f_scores[video_name] = f_score
        else:
            print(f"Advertencia: No se encontraron imágenes de sumarios humanos para el video '{video_name}'.")
            f_scores[video_name] = 0

    return f_scores

In [ ]:
# Obtener F-scores visuales para k-means con k=7, en ambas colecciones de datos
f_scores_visual_BDSumm_7 = calculate_visual_f_scores_from_images(key_frames_images_BDSumm_7, summaries_images_BDSumm, similarity_threshold=0.8)
f_scores_visual_OpenVideo_7 = calculate_visual_f_scores_from_images(key_frames_images_OpenVideo_7, summaries_images_OpenVideo, similarity_threshold=0.8)

# Obtener F-scores visuales para k-means con k=5, en ambas colecciones de datos
f_scores_visual_BDSumm_5 = calculate_visual_f_scores_from_images(key_frames_images_BDSumm_5, summaries_images_BDSumm, similarity_threshold=0.8)
f_scores_visual_OpenVideo_5 = calculate_visual_f_scores_from_images(key_frames_images_OpenVideo_5, summaries_images_OpenVideo, similarity_threshold=0.8)

# Obtener F-scores visuales para k-means con k=3, en ambas colecciones de datos
f_scores_visual_BDSumm_3 = calculate_visual_f_scores_from_images(key_frames_images_BDSumm_3, summaries_images_BDSumm, similarity_threshold=0.8)
f_scores_visual_OpenVideo_3 = calculate_visual_f_scores_from_images(key_frames_images_OpenVideo_3, summaries_images_OpenVideo, similarity_threshold=0.8)

Ahora observamos los f-scores visuales calculados:

In [ ]:
for video_name, f_score in f_scores_visual_OpenVideo_3.items():
    print(f"Video: {video_name}, F-score: {f_score}")

for video_name, f_score in f_scores_visual_OpenVideo_5.items():
    print(f"Video: {video_name}, F-score: {f_score}")

for video_name, f_score in f_scores_visual_OpenVideo_7.items():
    print(f"Video: {video_name}, F-score: {f_score}")

Video: v23, F-score: 0
Video: v50, F-score: 0.18181818181818182
Video: v59, F-score: 0
Video: v33, F-score: 0
Video: v68, F-score: 0.5
Video: v25, F-score: 0.125
Video: v46, F-score: 0.2
Video: v64, F-score: 0.11764705882352941
Video: v56, F-score: 0.3333333333333333
Video: v37, F-score: 0.5
Video: v39, F-score: 0
Video: v48, F-score: 0.2
Video: v54, F-score: 0
Video: v32, F-score: 0
Video: v60, F-score: 0
Video: v27, F-score: 0.2222222222222222
Video: v69, F-score: 0.30769230769230765
Video: v52, F-score: 0
Video: v21, F-score: 0
Video: v61, F-score: 0.3333333333333333
Video: v51, F-score: 0.2
Video: v42, F-score: 0.2
Video: v66, F-score: 0.36363636363636365
Video: v41, F-score: 0
Video: v62, F-score: 0
Video: v34, F-score: 0.4615384615384615
Video: v30, F-score: 0.14285714285714288
Video: v38, F-score: 0.14285714285714288
Video: v40, F-score: 0
Video: v57, F-score: 0.2222222222222222
Video: v65, F-score: 0.2
Video: v45, F-score: 0.2222222222222222
Video: v70, F-score: 0.2857142857142

Con los F-scores visuales calculados, se procede a determinar el mean F-score visual, es decir, el promedio de los F-scores visuales obtenidos.

In [ ]:
mean_f_score_visual_BDSumm_7 = calculate_mean_f_score(f_scores_visual_BDSumm_7)
mean_f_score_visual_BDSumm_5 = calculate_mean_f_score(f_scores_visual_BDSumm_5)
mean_f_score_visual_BDSumm_3 = calculate_mean_f_score(f_scores_visual_BDSumm_3)

mean_f_score_visual_OpenVideo_7 = calculate_mean_f_score(f_scores_visual_OpenVideo_7)
mean_f_score_visual_OpenVideo_5 = calculate_mean_f_score(f_scores_visual_OpenVideo_5)
mean_f_score_visual_OpenVideo_3 = calculate_mean_f_score(f_scores_visual_OpenVideo_3)

print(f"Mean F1-Score visual para k=7 (BDSumm): {mean_f_score_visual_BDSumm_7}")
print(f"Mean F1-Score visual para k=5 (BDSumm): {mean_f_score_visual_BDSumm_5}")
print(f"Mean F1-Score visual para k=3 (BDSumm): {mean_f_score_visual_BDSumm_3}")

print(f"Mean F1-Score visual para k=7 (OpenVideo): {mean_f_score_visual_OpenVideo_7}")
print(f"Mean F1-Score visual para k=5 (OpenVideo): {mean_f_score_visual_OpenVideo_5}")
print(f"Mean F1-Score visual para k=3 (OpenVideo): {mean_f_score_visual_OpenVideo_3}")

Mean F1-Score visual para k=7 (BDSumm): 0.14033572668655128
Mean F1-Score visual para k=5 (BDSumm): 0.09752496906582248
Mean F1-Score visual para k=3 (BDSumm): 0.08001704286584706
Mean F1-Score visual para k=7 (OpenVideo): 0.23467432876720792
Mean F1-Score visual para k=5 (OpenVideo): 0.21261803212267602
Mean F1-Score visual para k=3 (OpenVideo): 0.16523417105770047


Ahora, los resultados del F-score son superiores en comparación con la estrategia utilizada previamente. Esto indica que la incorporación de un enfoque más visual en el cálculo del F-score, basado en los fotogramas extraídos, mejora significativamente la alineación entre los sumarios automáticos y los humanos, lo que se traduce en una mayor precisión y recall.

Además, en ambas estrategias evaluadas, se observa que el valor óptimo de k es 7, ya que en este punto el F-score alcanza su valor más alto. Esto sugiere que para este conjunto de datos y el método empleado, k=7 representa una configuración ideal para los clusterings generados, maximizando la calidad del sumario automatizado.

# **2) Implementar dos baselines para obtener los sumarios estáticos de las colecciones BDSumm y OpenVideo. Los métodos baselines a implementar son: Random Sampling y Uniform Sampling.**

*   Random Sampling (RS), la idea es elegir K key-frames aleatorios para conformar el sumario.
*   Uniform Sampling (US), la idea es elegir K key-frames tomados de tiempos equidistantes en el video para conformar el sumario.

En primera instancia, se implementa el Random Sampling (RS), que selecciona k frames de cada video, aleatoriamente.

In [ ]:
import random

# Realiza Random Sampling para seleccionar k keyframes de cada video
def random_sampling(frames_dict, k):
    random_keyframes = {}

    for video_name, frames in frames_dict.items():
        total_frames = len(frames)
        if total_frames >= k:
            random_keyframes[video_name] = random.sample(range(total_frames), k)
        else:
            print(f"Advertencia: Video '{video_name}' tiene menos de {k} frames. Seleccionando todos.")
            random_keyframes[video_name] = list(range(total_frames))

    return random_keyframes

Después se implementa el Uniform Sampling (US), que selecciona k frames equidistantes de cada video.

In [ ]:
# Realiza Uniform Sampling para seleccionar k keyframes equidistantes de cada video.
def uniform_sampling(frames_dict, k):
    uniform_keyframes = {}

    for video_name, frames in frames_dict.items():
        total_frames = len(frames)
        if total_frames >= k:
            step = total_frames // k
            uniform_keyframes[video_name] = [i * step for i in range(k)]
        else:
            uniform_keyframes[video_name] = list(range(total_frames))

    return uniform_keyframes

Se utiliza el best-k (7) para aplicar el RS y el US en ambas colecciones de datos. En primera instancia se aplican ambos métodos en la colección de datos BDSumm.

In [ ]:
# Aplicar Random Sampling en DBSumm, con k=7 (best-k)
rs_keyframes_BDSumm = random_sampling(BDSumm_dict, k=7)
f_scores_rs_BDSumm = calculate_f_score_for_videos(rs_keyframes_BDSumm, normalized_summaries_BDSumm )

# Aplicar Uniform Sampling en DBSumm, con k=7 (best-k)
us_keyframes_BDSumm = uniform_sampling(BDSumm_dict, k=7)
f_scores_us_BDSumm = calculate_f_score_for_videos(us_keyframes_BDSumm, normalized_summaries_OpenVideo)

Advertencia: No se encontró un sumario humano para el video 'v19'
Advertencia: No se encontró un sumario humano para el video 'v15'
Advertencia: No se encontró un sumario humano para el video 'v11'
Advertencia: No se encontró un sumario humano para el video 'v12'
Advertencia: No se encontró un sumario humano para el video 'v20'
Advertencia: No se encontró un sumario humano para el video 'v16'
Advertencia: No se encontró un sumario humano para el video 'v18'
Advertencia: No se encontró un sumario humano para el video 'v13'
Advertencia: No se encontró un sumario humano para el video 'v17'
Advertencia: No se encontró un sumario humano para el video 'v14'


Después se aplican ambos métodos en la colección de datos OpenVideo

In [ ]:
# Aplicar Random Sampling en OpenVideo, con k=7 (best-k)
rs_keyframes_OpenVideo = random_sampling(OpenVideo_dict, k=7)
f_scores_rs_OpenVideo = calculate_f_score_for_videos(rs_keyframes_OpenVideo, human_summaries_OpenVideo)

# Aplicar Uniform Sampling en OpenVideo, con k=7 (best-k)
us_keyframes_OpenVideo = uniform_sampling(OpenVideo_dict, k=7)
f_scores_us_OpenVideo = calculate_f_score_for_videos(us_keyframes_OpenVideo, human_summaries_OpenVideo)

In [ ]:
# Obtener mean f-score para Random Sampling aplicado en BDSumm
mean_f_score_rs_BDSumm = calculate_mean_f_score(f_scores_rs_BDSumm)
# Obtener mean f-score para Uniform Sampling aplicado en BDSumm
mean_f_score_us_BDSumm = calculate_mean_f_score(f_scores_us_BDSumm)

print(f"Mean F1-Score para Random Sampling (BDSumm): {mean_f_score_rs_BDSumm}")
print(f"Mean F1-Score para Uniform Sampling (DBSumm): {mean_f_score_us_BDSumm}")

# Obtener mean f-score para Random Sampling aplicado en OpenVideo
mean_f_score_rs_OpenVideo = calculate_mean_f_score(f_scores_rs_OpenVideo)
# Obtener mean f-score para Random Sampling aplicado en OpenVideo
mean_f_score_us_OpenVideo = calculate_mean_f_score(f_scores_us_OpenVideo)

print(f"Mean F1-Score para Random Sampling (OpenVideo): {mean_f_score_rs_OpenVideo}")
print(f"Mean F1-Score para Uniform Sampling (OpenVideo): {mean_f_score_us_OpenVideo}")

Mean F1-Score para Random Sampling (BDSumm): 0.04492302333681645
Mean F1-Score para Uniform Sampling (DBSumm): 0.005
Mean F1-Score para Random Sampling (OpenVideo): 0.0036363636363636364
Mean F1-Score para Uniform Sampling (OpenVideo): 0.0


Los resultados continúan siendo muy bajos. Por esta razón, hemos decidido aplicar la estrategia alternativa: calcular el F-score visual.


In [ ]:
# Obtener frames a partir de los índices seleccionados por RS y US (BDSumm)
rs_key_frames_images_BDSumm = select_frames_from_indices(rs_keyframes_BDSumm, BDSumm_dict)
us_key_frames_images_BDSumm = select_frames_from_indices(us_keyframes_BDSumm, BDSumm_dict)

# Obtener f-score visual a partir de los frames seleccionados por RS y US (BDSumm)
rs_f_scores_visual_BDSumm = calculate_visual_f_scores_from_images(rs_key_frames_images_BDSumm, summaries_images_BDSumm, similarity_threshold=0.8)
us_f_scores_visual_BDSumm = calculate_visual_f_scores_from_images(us_key_frames_images_BDSumm, summaries_images_BDSumm, similarity_threshold=0.8)

In [ ]:
# Obtener frames a partir de los índices seleccionados por RS y US (OpenVideo)
rs_keyframes_images_OpenVideo = select_frames_from_indices(rs_keyframes_OpenVideo, OpenVideo_dict)
us_keyframes_images_OpenVideo = select_frames_from_indices(us_keyframes_OpenVideo, OpenVideo_dict)

# Obtener f-score visual a partir de los frames seleccionados por RS y US (OpenVideo)
rs_f_scores_visual_OpenVideo = calculate_visual_f_scores_from_images(rs_keyframes_images_OpenVideo, summaries_images_OpenVideo, similarity_threshold=0.8)
us_f_scores_visual_OpenVideo = calculate_visual_f_scores_from_images(us_keyframes_images_OpenVideo, summaries_images_OpenVideo, similarity_threshold=0.8)

Finalmente, podemos obtener los mean f-score visuales para las aplicaciones de RS y US en las colecciones de datos

In [ ]:
# Obtener mean f-score visual para Random Sampling aplicado en BDSumm
mean_f_score_rs_visual_BDSumm = calculate_mean_f_score(rs_f_scores_visual_BDSumm)
# Obtener mean f-score visual para Uniform Sampling aplicado en BDSumm
mean_f_score_us_visual_BDSumm = calculate_mean_f_score(us_f_scores_visual_BDSumm)

# Obtener mean f-score visual para Random Sampling aplicado en OpenVideo
mean_f_score_us_visual_OpenVideo = calculate_mean_f_score(us_f_scores_visual_OpenVideo)
# Obtener mean f-score visual para Random Sampling aplicado en OpenVideo
mean_f_score_rs_visual_OpenVideo = calculate_mean_f_score(rs_f_scores_visual_OpenVideo)

In [ ]:
print(f"Mean F1-Score visual para Random Sampling (BDSumm): {mean_f_score_rs_visual_BDSumm}")
print(f"Mean F1-Score visual para Uniform Sampling (BDSumm): {mean_f_score_us_visual_BDSumm}")

print(f"Mean F1-Score visual para Random Sampling (OpenVideo): {mean_f_score_rs_visual_OpenVideo}")
print(f"Mean F1-Score visual para Uniform Sampling (OpenVideo): {mean_f_score_us_visual_OpenVideo}")

Mean F1-Score visual para Random Sampling (BDSumm): 0.15667795128964543
Mean F1-Score visual para Uniform Sampling (BDSumm): 0.10639237926269411
Mean F1-Score visual para Random Sampling (OpenVideo): 0.1643499252725259
Mean F1-Score visual para Uniform Sampling (OpenVideo): 0.20578861798676037
